In [18]:
from main import *

In [19]:
info = read_file("sample1.dat")
pop = init_pop(2, **info)
best_solu = min(pop, key=lambda x: x.cost)
pop, best_solu

([{'cost': 29, 'routes': [{'remain_cap': 1, 'task_ids': [0, 2, 5, 0]}, {'remain_cap': 1, 'task_ids': [0, 1, 9, 0]}]},
  {'cost': 28, 'routes': [{'remain_cap': 1, 'task_ids': [0, 4, 1, 0]}, {'remain_cap': 2, 'task_ids': [0, 5, 8, 0]}]}],
 {'cost': 28, 'routes': [{'remain_cap': 1, 'task_ids': [0, 4, 1, 0]}, {'remain_cap': 2, 'task_ids': [0, 5, 8, 0]}]})

In [ ]:
def rev_id(id, num_tasks):
    """另一个方向的task id"""
    if id == 0:
        return 0
    return id + num_tasks if id <= num_tasks else id - num_tasks

def double(task_ids, num_tasks):
    """补全两个方向的task_id"""
    new_ids = [task_ids] + [rev_id(id, num_tasks) for id in task_ids]
    return new_ids

In [20]:
def crossover(sp, tasks, s1: Solution, s2: Solution):
    def eval(route: Route, id):
        idx = route.task_ids.index(id)
        v1 = tasks[route.task_ids[idx - 1]][0][1]
        v2 = tasks[route.task_ids[idx]][0][0]
        v3 = tasks[route.task_ids[idx]][0][1]
        v4 = tasks[route.task_ids[idx + 1]][0][0]
        return sp[v1, v2] + sp[v3, v4] - sp[v2, v3]
    
    s1r, s2r = s1.routes, s2.routes
    a, b = randint(0, len(s1r) - 1), randint(0, len(s2r) - 1)
    old, new = set(double(s1r[a].task_ids)), set(double(s2r[b].task_ids))
    s0r = s1r[:a] + s1r[a + 1:]  # 不加入 s2r[b]

    common = old & new
    unserved = old - common
    duplicated = new - common

    for id in duplicated:
        for r in s0r:
            if id in r.task_ids:
                # 判断应该移除哪一个
                eval1 = eval(s2r[b], id)
                eval2 = eval(r, id)

                if eval1 > eval2:
                    s2r[b].remove_id(id, tasks[id][1])
                else:
                    r.remove_id(id, tasks[id][1])
                
                duplicated.remove(id)
                duplicated.remove(rev_id(id))
                break
    
    s0r.append(s2r[b])  # 放入new
    for id in unserved:
        # 对于每一个没有被服务的task
        min_saving = MAX_COST
        insert_route, insert_idx = None, None
        for route in s0r:
            # 搜索每一条route，首先看看容量是否够
            if tasks[id][1] <= route.remain_cap:
                # 容量够了，遍历这个路线的所有位置，看看cost如何
                for idx in range(len(route.task_ids) - 1): # 不能插在最后，所以-1
                    v1 = tasks[route[idx]][0][1]
                    v2 = tasks[id][0][0]
                    v3 = tasks[id][0][1]
                    v4 = tasks[route[idx + 1]][0][0]
                    saving = sp[v1, v2] + sp[v3, v4] - sp[v2, v3]
                    if saving < min_saving:
                        min_saving = saving
                        insert_route, insert_idx = route, idx
        
        if insert_route is not None:
            # 找到了插入位置
            insert_route.insert(insert_idx, id)
        
    

                
        


In [21]:
crossover(info["sp"], info["tasks"], pop[0], pop[1])

TypeError: min expected at least 1 argument, got 0